In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('whitegrid')

In [25]:
df = pd.read_csv('train.csv')

In [26]:
df['word_count'] = df['comment_text'].apply(lambda x: len(str(x).split(" ")))

In [27]:
df['char_count'] = df['comment_text'].str.len() ## this also includes spaces


In [28]:
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

df['avg_word'] = df['comment_text'].apply(lambda x: avg_word(x))

In [29]:
from nltk.corpus import stopwords
import nltk


stop = stopwords.words('english')

df['stopwords'] = df['comment_text'].apply(lambda x: len([x for x in x.split() if x in stop]))

In [30]:
df['hastags'] = df['comment_text'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))

In [31]:
df['numerics'] = df['comment_text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df['upper'] = df['comment_text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))


In [32]:
df['comment_text'] = df['comment_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['comment_text'] = df['comment_text'].str.replace('[^\w\s]','')

In [33]:
df['comment_text'] = df['comment_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [34]:
freq = pd.Series(' '.join(df['comment_text']).split()).value_counts()[:30]

In [35]:
freq = list(freq.index)
df['comment_text'] = df['comment_text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [36]:
freq = pd.Series(' '.join(df['comment_text']).split()).value_counts()[-10:]
df['comment_text'] = df['comment_text'].apply(lambda x: " ".join(x for x in x.split() if x not in list(freq.index)))

In [37]:
df.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,word_count,char_count,avg_word,stopwords,hastags,numerics,upper
0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0,42,264,5.162791,14,0,0,2
1,000103f0d9cfb60f,daww matches background colour seemingly stuck...,0,0,0,0,0,0,18,112,5.588235,1,0,1,1
2,000113f07ec002fd,hey man really trying war guy constantly remov...,0,0,0,0,0,0,42,233,4.571429,18,0,0,0
3,0001b41b1c6bb37e,cant real suggestions improvement wondered sec...,0,0,0,0,0,0,112,622,4.486726,49,0,0,5
4,0001d958c54c6e35,sir hero chance remember thats,0,0,0,0,0,0,13,67,4.230769,4,0,0,0


In [38]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(df.drop(['toxic'],1),df['toxic'])

In [49]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train['comment_text'])
test_vectors = vectorizer.transform(X_test['comment_text'])
print(train_vectors.shape, test_vectors.shape)

(119678, 206206) (39893, 206206)


In [62]:
dtrain = lgb.Dataset(train_vectors,y_train)
dtest = lgb.Dataset(test_vectors,y_test)

param_lgb = {
    'max_depth': 5,
    'learning_rate': 0.33,
    'max_bin': 1000,
    'num_leaves' : 100,
    'objective': 'binary',
    'metric': 'auc',
    'num_iter': 1000
    
}


lgb_model = lgb.train(param_lgb,dtrain,valid_sets=dtest)

[1]	valid_0's auc: 0.670372
[2]	valid_0's auc: 0.725763
[3]	valid_0's auc: 0.751714
[4]	valid_0's auc: 0.767253
[5]	valid_0's auc: 0.786135
[6]	valid_0's auc: 0.802079
[7]	valid_0's auc: 0.808453
[8]	valid_0's auc: 0.824542
[9]	valid_0's auc: 0.836331
[10]	valid_0's auc: 0.847882
[11]	valid_0's auc: 0.856574
[12]	valid_0's auc: 0.862096
[13]	valid_0's auc: 0.865152
[14]	valid_0's auc: 0.869469
[15]	valid_0's auc: 0.873858
[16]	valid_0's auc: 0.875557
[17]	valid_0's auc: 0.882451
[18]	valid_0's auc: 0.888541
[19]	valid_0's auc: 0.891319
[20]	valid_0's auc: 0.892301
[21]	valid_0's auc: 0.893939
[22]	valid_0's auc: 0.896154
[23]	valid_0's auc: 0.897615
[24]	valid_0's auc: 0.898591
[25]	valid_0's auc: 0.899261
[26]	valid_0's auc: 0.902346
[27]	valid_0's auc: 0.903844
[28]	valid_0's auc: 0.906174
[29]	valid_0's auc: 0.907606
[30]	valid_0's auc: 0.909507
[31]	valid_0's auc: 0.912278
[32]	valid_0's auc: 0.913171
[33]	valid_0's auc: 0.914514
[34]	valid_0's auc: 0.916198
[35]	valid_0's auc: 0.9

In [64]:
dtrain = lgb.Dataset(X_train.drop(['comment_text','id'],1),y_train)
dtest = lgb.Dataset(X_test.drop(['comment_text','id'],1),y_test)

param_lgb = {
    'max_depth': 5,
    'learning_rate': 0.33,
    'max_bin': 1000,
    'num_leaves' : 100,
    'objective': 'binary',
    'metric': 'auc',
    'num_iter': 1000
    
}


lgb_model2 = lgb.train(param_lgb,dtrain,valid_sets=dtest)

[1]	valid_0's auc: 0.812351
[2]	valid_0's auc: 0.842407
[3]	valid_0's auc: 0.857714
[4]	valid_0's auc: 0.861177
[5]	valid_0's auc: 0.86424
[6]	valid_0's auc: 0.864351
[7]	valid_0's auc: 0.86574
[8]	valid_0's auc: 0.865757
[9]	valid_0's auc: 0.866805
[10]	valid_0's auc: 0.867401
[11]	valid_0's auc: 0.868065
[12]	valid_0's auc: 0.868169
[13]	valid_0's auc: 0.868145
[14]	valid_0's auc: 0.868682
[15]	valid_0's auc: 0.868597
[16]	valid_0's auc: 0.868544
[17]	valid_0's auc: 0.868422
[18]	valid_0's auc: 0.868388
[19]	valid_0's auc: 0.868177
[20]	valid_0's auc: 0.868292
[21]	valid_0's auc: 0.868499
[22]	valid_0's auc: 0.868216
[23]	valid_0's auc: 0.867978
[24]	valid_0's auc: 0.868106
[25]	valid_0's auc: 0.868033
[26]	valid_0's auc: 0.868169
[27]	valid_0's auc: 0.868138
[28]	valid_0's auc: 0.868418
[29]	valid_0's auc: 0.868568
[30]	valid_0's auc: 0.868503
[31]	valid_0's auc: 0.86847
[32]	valid_0's auc: 0.868316
[33]	valid_0's auc: 0.86839
[34]	valid_0's auc: 0.868318
[35]	valid_0's auc: 0.86836

In [65]:
res2 = lgb_model2.predict(X_test.drop(['comment_text','id'],1))

In [67]:
res1 = lgb_model.predict(test_vectors)

In [68]:
final = (np.array(res1) + np.array(res2)) / 2

In [70]:
pred = []
for i in final:
    pred.append(int(round(i)))

In [72]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,pred)

0.7998894641559168